In [1]:
#wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [2]:
#wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [3]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# labels for categories
names = { 0 : "fail", 1 : "success"}
plot_data = spacex_df['class'].value_counts()
plot_data.keys()

# list for dropdown options
sites = spacex_df["Launch Site"].unique()

site_options=[{'label': 'All Sites', 'value': 'ALL'}]
for site in sites:
    site_options.append({'label': site, 'value': site})

# Slider parameters 
Smin = 0
Smax = 10000
Sstep = 1000
                                
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                
                                dcc.Dropdown(id='site-dropdown',
                                        options=site_options,
                                        value='ALL',
                                        placeholder="place holder here",
                                        searchable=True
                                        ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=Smin, max=Smax, step=Sstep,
                                    marks={i : str(i) for i in range(Smin, Smax+Sstep, Sstep)},
                                    value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='total # of successful launches')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(["class"])["class"].count()
        fig = px.pie(filtered_df, values=filtered_df.values, 
        names=filtered_df.keys(), 
        title=f'lauches from {entered_site} by outcome')
        return fig
        # return the outcomes piechart for a selected site

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])
def get_scatter_plot(entered_site, payloadRange):
    Rmin, Rmax = payloadRange
    inPayloadRange = spacex_df[(spacex_df['Payload Mass (kg)'] >= Rmin) & (spacex_df['Payload Mass (kg)'] <= Rmax)]
    
    if entered_site != 'ALL':
        inPayloadRange = inPayloadRange[inPayloadRange['Launch Site'] == entered_site]
    
    fig = px.scatter(inPayloadRange, x = "Payload Mass (kg)", y = "class", color="Booster Version Category", 
    title=f'launch outcome by payload for {entered_site} site(s)')
    return fig
    
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\PieterMeijers\AppData\Local\Temp\ipykernel_13364\957385222.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\PieterMeijers\AppData\Local\Temp\ipykernel_13364\957385222.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [11/Jun/2023 22:29:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2023 22:29:33] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2023 22:29:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2023 22:29:33] "GET /_favicon.ico?v=2.9.3 HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2023 22:29:33] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [11/Jun/2023 22:29:33